### Training method: Neural Network, XGBoost, LGBoosting, Liniar Regression
### The most important features are: 

### How long it takes to train your model

In [ ]:
pip freeze

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime
from workalendar.europe import Russia
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import gc
from itertools import product
from datetime import date
import time

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LinearRegression, BayesianRidge, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from xgboost import XGBRegressor
from xgboost import plot_importance

from keras.layers import Input, Dense, Dropout
from keras.models import Model
from keras import optimizers
from keras.models import load_model

In [ ]:
def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

def mean_enc(all_data, group_cols, target_col, name, fill_c):
    gb = all_data[group_cols + [target_col]].groupby(group_cols,as_index=False).agg({target_col:{name + '_mean':'mean'}})
    #gb.reset_index(inplace=True)
    gb.columns = [col[0] if col[-1] == '' else col[-1] for col in gb.columns.values]
    all_data = pd.merge(all_data, gb, how='left', on=group_cols).fillna(fill_c)
    return all_data
  
def mean_KFenc(data, enc_col, target_col, n_splits):
    data= data.copy()
    y= data[target_col]; X= data[enc_col]
    new_col= enc_col + '_mean_skf'
    data[new_col]= fill_c
    skf = StratifiedKFold(n_splits=n_splits, shuffle= True, random_state=0)
    for train_index, test_index in skf.split(X, y):
        enc= data.loc[train_index].groupby(enc_col)[target_col].mean()
        data[new_col].loc[test_index]= data[enc_col].loc[test_index].map(enc)    
    data[new_col].fillna(data[new_col].mean(), inplace=True)
    return data


def mean_expand_enc(all_data, enc_col, target_col):
    #gb= all_data.groupby(enc_col)[target_col]
    new_name= enc_col + '_expand_mean'
    all_data['cs']= all_data.groupby(enc_col)[target_col].cumsum() - all_data[target_col]
    all_data['ct']= all_data.groupby(enc_col)[target_col].cumcount()
    all_data[]= all_data['cs'] / all_data['ct']
    all_data[new_name].fillna(all_data[new_name].mean(), inplace=True)
    return all_data



def feature_lags(all_data, time_col, inx_cols, name, lags, fill_c):  
    for lag in lags:
        d= all_data[inx_cols + [name]].copy().drop_duplicates(subset=inx_cols)
        d[time_col] += lag
        d= d.rename(columns= {name: name + '_previous_' + str(lag)})
        all_data = pd.merge(all_data, d, how='left', on=inx_cols).fillna(fill_c)
    del d
    return all_data

def first_last(all_data, p_cols, reference, name, fl):
    gb= all_data[p_cols + [reference]].groupby(p_cols, as_index=False)
    #gb.reset_index(inplace=True)
    if fl== 'first': 
        a=gb.first()
    if fl== 'last':
        a=gb.last()
    a= a.rename(columns={reference:fl + '_' + name})
    all_data = pd.merge(all_data, a, how='left', on=p_cols)
    if fl== 'first': 
        all_data['delta_' + fl + '_' + name]= (all_data[reference] - all_data[fl + '_' + name]).fillna(0)
    if fl== 'last': 
        all_data['delta_' + fl + '_' + name]= (all_data[reference] - all_data[fl + '_' + name]).fillna(33)
    return all_data

fill_c= 0

In [ ]:
#read_data
path= 'Data/'
t          = pd.read_csv(path + 'sales_train.csv.gz')
test            = pd.read_csv(path + 'test.csv')
items           = pd.read_csv(path + 'items.csv')
item_cats = pd.read_csv(path + 'item_categories.csv')
shops           = pd.read_csv(path + 'shops.csv')

t['date']= pd.to_datetime(t['date'], format= '%d.%m.%Y')
t= t.sort_values('date')

# Input data EDA

In [ ]:
t.columns

In [ ]:
shops.columns

In [ ]:
test.columns

## Train-Test differences

In [ ]:
# Checking whether there ares new items in test data.
col= 'item_id'
new_items=len(list(set(test[col]) - set(test[col]).intersection(set(t[col]))))

train_items= t[col].unique()
test_items= test[col].unique()
print('Number of train items: ', len(train_items))
print('Number of test items: ', len(test_items))
print('Number of new items in test: ', new_items)

In [ ]:
# Checking wether there are new shops in test data.
col= 'shop_id'
new_shops=set(test[col]).difference(set(t[col]))
new_shops

In [ ]:
difference_shops=set(t[col]).difference(set(test[col]))
difference_shops

In [ ]:
train_shops= sorted(t[col].unique())
test_shops= sorted(test[col].unique())
print('train shops: ', train_shops)
print('test shops: ', test_shops)

## Shops
#### There seems to be a duplications of shops 0 with 57, 1 with 58 & 10 with 11. Since that shops 0, 1 & 11 aren't appearing in test data, then 0 should be convrted to 57, 1 to 58 & 10 to 11.

In [ ]:
gb = shops.groupby('shop_name')
 
gb.head()

## items

In [ ]:
items.columns

In [ ]:
gb = items.groupby('item_name')


In [ ]:
gb.head()

## item_category

In [ ]:
item_cats.head(10)

# Clean

In [ ]:
# Cleaning and ordering based on EDA

# removing outliers
t = t[t['item_price']<50000]
t = t[t['item_cnt_day']<500]

# 

t.loc[t['shop_id'] == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57

t.loc[t['shop_id'] == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58

t.loc[t['shop_id'] == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

shops.loc[shops.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'
shops['city'] = shops['shop_name'].str.split(' ').map(lambda x: x[0])
shops.loc[shops.city == '!Якутск', 'city'] = 'Якутск'
shops['city_code'] = LabelEncoder().fit_transform(shops['city'])
shops = shops[['shop_id','city_code']]

items.drop(['item_name'], axis=1, inplace=True)



item_cats['split'] = item_cats['item_category_name'].str.split('-')
item_cats['type'] = item_cats['split'].map(lambda x: x[0].strip())
item_cats['type_code'] = LabelEncoder().fit_transform(item_cats['type'])
# if subtype is nan then type
item_cats['subtype'] = item_cats['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
item_cats['subtype_code'] = LabelEncoder().fit_transform(item_cats['subtype'])
item_cats = item_cats[['item_category_id','type_code', 'subtype_code']]

# Check leakedges

In [ ]:
plt.scatter(t.index, t['item_cnt_day'], s=0.5); plt.show()

In [ ]:
plt.scatter(t['shop_id'], t['item_cnt_day'], s=0.5); plt.show()

In [ ]:
plt.scatter(test['shop_id'], test['ID'], s=0.5);plt.show()

In [ ]:
plt.figure(figsize=(15,15))
plt.scatter(test['shop_id'], test['item_id'], s=0.5);plt.show()

In [ ]:
plt.figure(figsize=(15,15))
plt.scatter(test['item_id'], test['ID'], s=0.5);plt.show()

#### There are some patterns that require further investigation in the future. 

In [ ]:
tr_shops= list(t['shop_id'].unique())
ts_shops= list(test['shop_id'].unique())
tr_items= list(t['item_id'].unique())
ts_items= list(test['item_id'].unique())

# Create basic dataset for processing
#### For each item in each shop I create an instance for each month. This way there will be a continious data of each specific item for creating features and training the model.  
#### Test data will be concatenated to train data for easier processing.

In [ ]:
# Create grid
index_cols = ['shop_id', 'item_id', 'date_block_num']

grid = [] 
for block_num in t['date_block_num'].unique():
    cur_shops = t.loc[t['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = t.loc[t['date_block_num'] == block_num, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

# Turn the grid into a dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols, dtype=np.int32)


grid['ID']= -1

#grid= grid[grid['shop_id'].isin(ts_shops)]
#grid= grid[grid['item_id'].isin(ts_items)]


g= grid.shape
print('grid', g)


# Preparing test set
test['date_block_num'] = 34
test['date_block_num'] = test['date_block_num'].astype(np.int8)
test['shop_id'] = test['shop_id'].astype(np.int8)
test['item_id'] = test['item_id'].astype(np.int16)

keep= len(grid)+len(test)
print('keep: ', keep)

In [ ]:
# Concatenating grid & test data
all_data= pd.concat([grid, test], axis=0, ignore_index=True)#, sort=False)
all_data.fillna(0, inplace=True)
del grid

# Merging Shops, Categories, items 
all_data = pd.merge(all_data, shops, on=['shop_id'], how='left')
all_data = pd.merge(all_data, items, on=['item_id'], how='left')
all_data = pd.merge(all_data, item_cats, on=['item_category_id'], how='left')
all_data['item_category_id'] = all_data['item_category_id'].astype(np.int8)
all_data['city_code'] = all_data['city_code'].astype(np.int8)
all_data['type_code'] = all_data['type_code'].astype(np.int8)
all_data['subtype_code'] = all_data['subtype_code'].astype(np.int8)

print(all_data.shape)

### Calculate target data
#### I will clip target to 0-20 for better convergance of the model.

In [ ]:
# Target of current month
group_cols= ['shop_id', 'item_id', 'date_block_num']
gb = t.groupby(group_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=group_cols).fillna(0)

# Clipping
all_data['target']= all_data['target'].clip(0, 20, axis=0).astype(np.int8)
print(all_data.shape)

# Features
#### * Months & year.
#### * Months since firs & last sales.
#### * Mean price for each month, price lags & trends.
#### * Working days and holydays in each month.
#### * Mean encoding using skf strategy.
#### * Target lags, trends & estimations.

In [ ]:
 '''
More to investigate: 
1. Shops location relative to other locations. Same as in video.
2. Text from item_name. need to encode by characteristics, like BD, D, etc..
3. TF-IDF.
4. Model for new features. For example those which not appear at the first dbn.
5. Analyze sales along years. find the average factor between sales.
6. investigate encoding to pairs and with inx_cols
'''

In [ ]:
# Calculate month & year
all_data['month']= (all_data['date_block_num'] % 12 + 1).astype(np.int32)
all_data['year']= (all_data['date_block_num'] // 12 + 2013).astype(np.int32)

# months since first & last sale
reference= 'date_block_num'

p_cols=['shop_id', 'item_id']; name= 'shop_item'; fl= 'first'
all_data= first_last(all_data, p_cols, reference, name, fl)

p_cols=['shop_id', 'item_id']; name= 'shop_item'; fl= 'last'
all_data= first_last(all_data, p_cols, reference, name, fl)

p_cols=['shop_id']; name= 'shop'; fl= 'first'
all_data= first_last(all_data, p_cols, reference, name, fl)

p_cols=['shop_id']; name= 'shop'; fl= 'last'
all_data= first_last(all_data, p_cols, reference, name, fl)

p_cols=['item_id']; name= 'item'; fl= 'first'
all_data= first_last(all_data, p_cols, reference, name, fl)

p_cols=['item_id']; name= 'item'; fl= 'last'
all_data= first_last(all_data, p_cols, reference, name, fl) 

In [ ]:
# Prices
time_col= 'date_block_num'
inx_cols= ['shop_id', 'item_id', 'date_block_num']
lags= [1, 2, 3, 4, 6, 9, 12]
gb= t[inx_cols + ['item_price']].groupby(inx_cols, as_index=False).mean()
all_data= pd.merge(all_data, gb, how= 'left', on=inx_cols) 
all_data= feature_lags(all_data, time_col, inx_cols, 'item_price', lags, fill_c)
for l in lags[1:]:
    all_data['item_price_trend_' + str(l)]= all_data['item_price_previous_1'] - all_data['item_price_previous_' + str(l)]
all_data.drop('item_price', axis=1, inplace=True)
print('prices')

In [ ]:
#Time features    
days_in_month = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
all_data['month'] -=1
all_data['days'] = all_data['month'].map(days_in_month).astype(np.int8)
all_data['month'] +=1
cal = Russia()

y= 2014
all_data['w_days']= 0

for m, last in enumerate(days_in_month):
    all_data['w_days'][all_data['month']==m+1]= cal.get_working_days_delta(date(y, m+1, 1), date(y, m+1, last))
all_data['h_days']= all_data['days'] - all_data['w_days'] - 8
gc.collect();
print(all_data.shape)


all_data['date_block_num'] += 1

print('time')
print(all_data.shape)

In [ ]:
# Expand Mean Encoding  
enc_cols= ['shop_id', 'item_id', 'item_category_id', 'type_code', 'subtype_code', 'city_code']
#inx_cols= ['shop_id', 'item_id', 'date_block_num']
target_col= 'target'

for enc_col in enc_cols:
    all_data= mean_expand_enc(all_data, enc_col, target_col)
    
all_data.drop(['cs', 'ct'], axis=1, inplace=True)

In [ ]:
# SKF Mean Encoding    
inx_cols= ['shop_id', 'item_id', 'date_block_num']
time_col= 'date_block_num'
target_col= 'target'

skf_cols= ['shop_id', 'item_id', 'item_category_id', 'type_code', 'subtype_code', 'city_code']#, 'month', 'year']
data1= all_data[all_data['ID']==-1]
data2= all_data[all_data['ID']>-1]
for skf_col in skf_cols:
    data1= mean_KFenc(data1, skf_col, target_col, n_splits=10)
    data2= pd.merge(data2, data1[[skf_col, skf_col + '_mean_skf']].drop_duplicates(subset=skf_col, keep='last'), how='left', on=skf_col)
all_data= pd.concat([data1, data2], axis=0)

print('skf_enc')
print('all_data shape: ', all_data.shape)




In [ ]:
# Pairs Mean Encoding
group_cols= ['shop_id', 'item_id']; name= 'target_shop_item'
all_data= mean_enc(all_data, group_cols, target_col, name, fill_c)

group_cols= ['shop_id', 'city_code']; name= 'target_shop_city'
all_data= mean_enc(all_data, group_cols, target_col, name, fill_c)

group_cols= ['shop_id', 'item_category_id']; name= 'target_shop_cat'
all_data= mean_enc(all_data, group_cols, target_col, name, fill_c)

group_cols= ['shop_id', 'type_code']; name= 'target_shop_type'
all_data= mean_enc(all_data, group_cols, target_col, name, fill_c)

group_cols= ['shop_id', 'subtype_code']; name= 'target_shop_subtype'
all_data= mean_enc(all_data, group_cols, target_col, name, fill_c)

print('mean encoding')
print('all_data shape: ', all_data.shape)

In [ ]:
# Target lags
group_cols= ['shop_id', 'item_id', 'date_block_num']
lags= [1, 2, 3, 4, 5, 6, 9, 12]
for i in lags:       
    d= all_data[group_cols + ['target']].copy().drop_duplicates()
    d['date_block_num'] += i
    d= d.rename(columns= {'target':'target_previous_' + str(i)})
    all_data = pd.merge(all_data, d, how='left', on=group_cols).fillna(0)
del d
print('target lags')
print('all_data shape: ', all_data.shape)

# Target trend
inx_cols= ['shop_id', 'item_id', 'date_block_num']
time_col= 'date_block_num'
target_col= 'target'
trend_lags= [2, 3, 4, 5, 6, 9, 12]
for l in trend_lags:
    all_data['target_trend_' + str(l)]= all_data['target_previous_1'] - all_data['target_previous_' + str(l)]
    all_data['target_trend_' + str(l)].fillna(-1)
name= 'target_trend_2'

all_data= feature_lags(all_data, time_col, inx_cols, name, [11, 12, 13], fill_c)

print('target trends')
print('all_data shape', all_data.shape)

#Trying to estimate
all_data['tar11']= all_data['target_previous_1'] * (1 + all_data['target_trend_2_previous_11'] / all_data['target_previous_12']).replace([np.inf, -np.inf, np.nan], 1)
all_data['tar12']= all_data['target_previous_1'] * (1 + all_data['target_trend_2_previous_12'] / all_data['target_previous_12']).replace([np.inf, -np.inf, np.nan], 1)
all_data['tar13']= all_data['target_previous_1'] * (1 + all_data['target_trend_2_previous_13'] / all_data['target_previous_12']).replace([np.inf, -np.inf, np.nan], 1)
all_data['tar111']= all_data['target_previous_1'] + all_data['target_trend_2_previous_11']
all_data['tar112']= all_data['target_previous_1'] + all_data['target_trend_2_previous_12']
all_data['tar113']= all_data['target_previous_1'] + all_data['target_trend_2_previous_13']
    
print('estimate')
print('all_data shape: ', all_data.shape)
    

In [ ]:
all_data = downcast_dtypes(all_data)
print('all_data shape: ', all_data.shape);
print(all_data.info())

In [ ]:
all_data.to_csv('all_data.csv', index=False);

# Analyze features

In [ ]:
'''
More to investigate: 
1. Analyze differences between features distribution in train & test datas.
'''

In [ ]:
cut= 14
train_data= all_data[all_data['ID']==-1]

y= train_data[['target', 'date_block_num']][train_data['date_block_num'] >= cut]

X= train_data.drop(['ID', 'target'], axis=1)
X= X[X['date_block_num'] >= cut]
dbn= X['date_block_num']
X= X.drop('date_block_num', axis=1)


test_data=  all_data[all_data['ID'] > -1].drop(['target', 'date_block_num'], axis=1)
ID= test_data['ID']
test_data.drop('ID', axis=1, inplace=True)

del all_data

In [ ]:
cols= ['target_shop_item_mean', 'target_shop_city_mean', 'target_shop_cat_mean','target_shop_type_mean', 
       'target_shop_subtype_mean', 'item_id', 'shop_id', 'city_code', 'item_category_id', 'type_code',
       'subtype_code', 'month', 'year', 'item_id_mean_skf', 'shop_id_mean_skf']
X= X.drop(cols, axis=1)
test_data= test_data.drop(cols, axis=1)

In [ ]:
X.columns

In [ ]:
plt.scatter(t['item_id'], t['item_price'], s=0.5); plt.show()

# Advanced features
#### * Selecting data from month 14
#### * Split between data for modeling and data for submission.
#### * Extract RF leaves.

In [ ]:
# RF features
RFmodel = RandomForestRegressor(max_depth=3, min_samples_split=1000, random_state=0,n_estimators=3)
RFmodel.fit(X, y)

a= RFmodel.apply(X)
b= RFmodel.apply(test_data)

RFX= pd.DataFrame(data=a, columns=['RF1', 'RF2', 'RF3'])
RFtest= pd.DataFrame(data=b, columns=['RF1', 'RF2', 'RF3'])

feature_importances = pd.DataFrame(RFmodel.feature_importances_, 
                                   index = X.columns, 
                                   columns=['importance']).sort_values('importance', ascending=False)
feature_importances

In [ ]:
feature_importances

In [ ]:
RFX.to_csv('RFX.csv')
RFtest.to_csv('RFtest.csv')

In [ ]:
cols= X.columns.tolist()
scaler = MinMaxScaler()
#scaler = StandardScaler()
XS= scaler.fit_transform(X)
test_dataS= scaler.transform(test_data)

scaler1 = MinMaxScaler(feature_range=(0, 1))
yS= scaler1.fit_transform(y.values.reshape(-1, 1))


XS= pd.DataFrame(data=XS, columns=cols)
test_dataS= pd.DataFrame(data=test_dataS, columns=cols)
yS= pd.DataFrame(data=yS, columns=['target'])

In [ ]:
KNNmodel= KNeighborsRegressor(n_neighbors=5)
KNNmodel.fit(XS, y)
distX= KNNmodel.kneighbors(XS)
dist_test= KNNmodel.kneighbors(test_dataS)

knncols=[]
for i in distX.shape[1]:
    knncols.append('knn5' + str(i))
    
distX= pd.DataFrame(data=distX, columns= knncols)
dist_test= pd.DataFrame(data=dist_test, columns=knncols)
print('knn')

In [ ]:
distX.to_csv('distX.csv')
dist_test.to_csv('dist_test.csv')

# Pre-Training
#### * Split between train and validation data.
#### * Modeling with Light Gradient Boosting.

In [ ]:
X= pd.read_csv('X_data.csv');
y= pd.read_csv('y_data.csv');
test_data= pd.read_csv('test_data.csv');
dbn= pd.read_csv('date_block_num.csv')
RFX= pd.read_csv('RFX.csv')
RFtest= pd.read_csv('RFtest.csv')
#distX= pd.read_csv('distX.csv')
#dist_test= pd.read_csv('dist_test.csv')

In [ ]:
X= pd.read_csv('X_data.csv');
y= pd.read_csv('y_data.csv');
test_data= pd.read_csv('test_data.csv');

In [ ]:
Xrf= pd.concat([XS, RFX], axis=1)
test_data_rf= pd.concat([test_dataS, RFtest], axis=1)

In [ ]:
def ens_models(X_train, y_train, X_valid):


    tlevel1= []; vlevel1= []
    print('lgb')
    lgb_params1 = {'feature_fraction': 0.7,'metric': 'rmse', 'nthread':12, 'min_data_in_leaf': 2**15, 
                      'bagging_fraction': 0.7, 'learning_rate': 0.05, 'objective': 'rmse',
                      'bagging_seed': 2**9, 'num_leaves': 2**9,'bagging_freq':1,'verbose':0}

    lgb_params2 = {'feature_fraction': 0.7,'metric': 'rmse', 'nthread':12, 'min_data_in_leaf': 2**11, 
                  'bagging_fraction': 0.7, 'learning_rate': 0.07, 'objective': 'rmse',
                  'bagging_seed': 2**7, 'num_leaves': 2**9,'bagging_freq':1,'verbose':0}

    lgb_params3 = {'feature_fraction': 0.7,'metric': 'rmse', 'nthread':12, 'min_data_in_leaf': 2**7, 
                  'bagging_fraction': 0.7, 'learning_rate': 0.05, 'objective': 'rmse',
                  'bagging_seed': 2**5, 'num_leaves': 2**11,'bagging_freq':1,'verbose':0}

    for n, lgb_params in enumerate ([lgb_params1, lgb_params2, lgb_params3]):
        model = lgb.train(lgb_params, lgb.Dataset(X_train, label=y_train), 30)
        #ax = lgb.plot_importance(model, figsize=(15, 15));plt.show()
        
        t_p= model.predict(X_train)
        v_p= model.predict(X_valid)
        vlevel1.append(v_p)
    
    print('RF')
    RF1= RandomForestRegressor(max_depth= 3, min_samples_split= 1000, min_samples_leaf= 50, random_state= 0, n_estimators= 10)
   
    for n, RFmodel in enumerate ([RF1]):
        RFmodel.fit(X_train, y_train)
     
    
        t_p= model.predict(X_train)
        tlevel1.append(t_p)
        v_p= RFmodel.predict(X_valid)
        vlevel1.append(v_p)
    
    print('LR')    
    LRmodel= LinearRegression()
    Lmodel= Lasso()
    Rmodel= Ridge()
    BRmodel= BayesianRidge()
    for n, model in enumerate ([LRmodel, Lmodel, Rmodel, BRmodel]):
        model.fit(X_train, y_train)
    
        t_p= model.predict(X_train)
        tlevel1.append(t_p.squeeze())
        v_p= model.predict(X_valid)
        vlevel1.append(v_p.squeeze())
    
    return vlevel1, tlevel1

In [ ]:
XS.columns

In [ ]:
predicted_train= Xrf
predicted_train['date_bulk']= dbn.values.squeeze()
predicted_test= test_data_rf
yy= y.copy()
yy.columns= ['target', 'date_bulk']
#yy['date_bulk']= dbn.values.squeeze()
last_bulk= predicted_train['date_bulk'].max()

for n, k in enumerate ([30, last_bulk-1, last_bulk]): #[30, last_bulk-1, last_bulk]
    print(k)
    XX= predicted_train.copy()
    df_test= predicted_test.copy()

    predicted_train= pd.DataFrame()
    #k= 24    
    for m in range(k, last_bulk + 1):
        print(m)
        chunk= np.arange(XX['date_bulk'].min(), m + 1)
        X_train= XX[XX['date_bulk'].isin(chunk)]
        y_train= yy[yy['date_bulk'].isin(chunk)]

        if m < last_bulk:
            X_valid= XX[XX['date_bulk']== m + 1]

            vlevel1, _= ens_models(X_train.drop('date_bulk', axis=1), y_train.drop('date_bulk', axis=1), X_valid.drop('date_bulk', axis=1))

            df= pd.DataFrame(data= np.array(vlevel1).T, columns= np.arange(1, len(vlevel1) + 1))
            df['date_bulk']= m + 1

            predicted_train= pd.concat([predicted_train, df], axis=0)
        else:
            X_valid= df_test

            vlevel1, tlevel1= ens_models(X_train.drop('date_bulk', axis=1), y_train.drop('date_bulk', axis=1), X_valid)
            vlevel1= np.array(vlevel1).T
            tlevel1= np.array(tlevel1).T
            predicted_test= pd.DataFrame(data= vlevel1, columns= np.arange(1, vlevel1.shape[1] + 1))


BRmodel1= BayesianRidge()
BRmodel1.fit(tlevel1, y_train.drop('date_bulk', axis=1))
pr1= BRmodel1.predict(tlevel1).clip(0,20)
pr2= BRmodel1.predict(vlevel1).clip(0,20)
            
print('dbn 34 before final level (mean) rmse: ', np.sqrt(mean_squared_error(y['target'][yy['date_bulk']==34], df.drop('date_bulk', axis=1).mean(axis=1).clip(0,20))));
print('dbn 34 before final level rmse: ', np.sqrt(mean_squared_error(y['target'][yy['date_bulk']==34], df[4].clip(0,20))));

print('all modeling data rmse: ', np.sqrt(mean_squared_error(y['target'].iloc[-len(pr1):], pr1)));
print('dbn 34 rmse: ', np.sqrt(mean_squared_error(y['target'][yy['date_bulk']==34], pr1[-len(y['target'][yy['date_bulk']==34]):])));

In [ ]:
d= df.copy()
d['target']= y['target'][yy['date_bulk']==34].values
d.head(40)

# Pridiction

In [ ]:
test_data['ID']= ID.values
ID= test_data[['ID']];
ID['ID']= ID['ID'].astype(int) 

#ID['item_cnt_month']= predicted_test.mean(axis=1).values;
#ID['item_cnt_month']= predicted_test[3].values; #!!!!!
ID['item_cnt_month']= pr2
ID['item_cnt_month']= ID['item_cnt_month'].clip(0,20)
ID.head(10)

In [ ]:
ID['item_cnt_month'].max()

In [ ]:
ID.to_csv('SUBMISSION07MAY_0.csv', index=False)